In [1]:
import os
import glob
import psycopg2 as pg
from psycopg2 import extras
from psycopg2 import sql
import pandas as pd  
import nltk
import re
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sqlalchemy import create_engine
import swifter

sid = SentimentIntensityAnalyzer()
import datetime
exitFlag = 0

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/docker_worker/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
config = {
    'host' : 'postgres',
    'user' : 'postgres',
    'password' : 'tuturu123',
    'dbname' : 'pdt2021_tweets'
}

conn = pg.connect(**config)
cursor = conn.cursor('cursor')
tableName = 'test_table'

queryText = '''select DISTINCT tw.id, tw.content from hashtags as h 
	INNER join tweet_hashtags as th 	on th.hashtag_id = h.id
	INNER join tweets as tw 		    on tw.id = th.tweet_id
	WHERE h.value ilike any(array['%%DeepstateVirus%%',
'%%DeepStateVaccine%%',
'%%DeepStateFauci%%',
'%%QAnon%%',
'%%Agenda21%%',
'%%CCPVirus%%',
'%%ClimateChangeHoax%%',
'%%GlobalWarmingHoax%%',
'%%ChinaLiedPeopleDied%%',
'%%SorosVirus%%',
'%%5GCoronavirus%%',
'%%MAGA%%',
'%%WWG1WGA%%',
'%%Chemtrails%%',
'%%flatEarth%%',
'%%MoonLandingHoax%%',
'%%moonhoax%%',
'%%illuminati%%',
'%%pizzaGateIsReal%%',
'%%PedoGateIsReal%%',
'%%911truth%%',
'%%911insidejob%%',
'%%reptilians%%' 
]);'''


a = datetime.datetime.now()
print("Zaciatok vyberu")
cursor.execute(queryText)
record = cursor.fetchall()
print("Koniec vyberu")
b = datetime.datetime.now()
c = b - a
print(c.seconds, "seconds")
print("Zaciatok prehadzovania na DF")
df = pd.DataFrame(record, columns=[
'id',
'content'
])
print("Koniec prehadzovania na DF")


Zaciatok vyberu
Koniec vyberu
193 seconds
Zaciatok prehadzovania na DF
Koniec prehadzovania na DF


In [ ]:
def remove_shit(text):
    # remove hastags
    hastag_pattern = re.compile("#([a-z-A-Z])\w+", flags=re.UNICODE)
    text = hastag_pattern.sub(r'', text)
    # remove metions
    mentions_pattern = re.compile("@([a-z-A-Z])\w+", flags=re.UNICODE)
    text = mentions_pattern.sub(r'', text)
    # remove emotikons
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text

def calculate_sentiment(text):
    sentiment = sid.polarity_scores(text)
    return sentiment
    

def update_vals(row):
    text = row['content']
    text = remove_shit(text)
    row['content'] = text
    sentiment = calculate_sentiment(text)
    row['neg'] = sentiment['neg']
    row['neu'] = sentiment['neu']
    row['pos'] = sentiment['pos']
    row['compound'] = sentiment['compound']      
    return row

df = df.swifter.apply(update_vals, axis=1)

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

In [23]:
# df = df.drop(['test'], axis = 1)

print(df.count)

<bound method DataFrame.count of                           id  \
0        1034289685227614208   
1        1037494012033343488   
2        1038674252940148736   
3        1039279051456831489   
4        1039284206856286208   
...                      ...   
2152419   984572021303664640   
2152420   986043842603507712   
2152421   988105922831667200   
2152422   997954448407351296   
2152423   998346648597221376   

                                                   content  
0        PEDOPHILIA IS AN EPIDEMIC IN USA-WORLD: \n\nSo...  
1        \nPEDOPHILIA ‘EPIDEMIC’ WORLD-UNTOUCHABLES:\n\...  
2        and , Wary of , Agree to Strengthen Military T...  
3        COMMUNlST CHlNA CENSORSHIP IN AMERICA: \n\nNYT...  
4        China owns our dept. The Chinese Gov. are work...  
...                                                    ...  
2152419   \nPedophilia Epidemic World: \n\nJimmy Savile...  
2152420   \n \n \n \n\nThread Within -\nhttps://t.co/A9...  
2152421   \nCHILD TRAFFICKING: \n

In [ ]:

cursor.close()
conn.close()


In [149]:
df = pd.read_sql_query('''select * from test_table where content like '%%#%%' limit 100;''', conn)

In [150]:
print(df.loc[0])

index                                                             0
id                                              1223809654129512453
content           BREAKING: A Chinese in the Philippines, which ...
location                                                       None
retweet_count                                                    32
favorite_count                                                   25
happened_at                               2020-02-02 03:25:33+00:00
author_id                                        911209149844688898
country_id                                                     None
parent_id                                                      None
pos                                                             9.0
Name: 0, dtype: object


In [179]:
def remove_shit(text):
    # remove hastags
    hastag_pattern = re.compile("#([a-z-A-Z])\w+\s+", flags=re.UNICODE)
    text = hastag_pattern.sub(r'', text)
    # remove metions
    mentions_pattern = re.compile("@([a-z-A-Z])\w+:\s+", flags=re.UNICODE)
    text = mentions_pattern.sub(r'', text)
    # remove emotikons
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text

for frame in df.iloc:
    frame.content = remove_shit(frame.content)    



In [127]:
df.loc[0, 'favorite_count'] = 25
df.loc[0, 'pos'] = 9 
print(df.loc[0])

id                                              1223809654129512453
content           BREAKING: A Chinese in the Philippines, which ...
location                                                       None
retweet_count                                                    32
favorite_count                                                   25
happened_at                               2020-02-02 03:25:33+00:00
author_id                                        911209149844688898
country_id                                                     None
parent_id                                                      None
pos                                                             9.0
Name: 0, dtype: object


In [128]:
df.to_sql('test_table', conn, 'public', 'replace')

In [129]:

df_new = pd.read_sql_query('''select * from test_table t limit 10;''', conn)
print(df_new.loc[0])

index                                                             0
id                                              1223809654129512453
content           BREAKING: A Chinese in the Philippines, which ...
location                                                       None
retweet_count                                                    32
favorite_count                                                   25
happened_at                               2020-02-02 03:25:33+00:00
author_id                                        911209149844688898
country_id                                                     None
parent_id                                                      None
pos                                                             9.0
Name: 0, dtype: object
